# YOLOv9 for pose estimation 

First we neede to convert all of the data we'vr created into yolo dataset.
We did it in coco_to_yolo.py

In [ ]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
%pip install ultralytics opencv-python matplotlib pyyaml

In [9]:
from ultralytics.utils.checks import check_yolo
print(check_yolo())

Ultralytics 8.3.179 🚀 Python-3.10.18 torch-2.5.1+cu121 CUDA:0 (Tesla V100-PCIE-16GB, 16144MiB)
Setup complete ✅ (6 CPUs, 110.0 GB RAM, 122.0/145.2 GB disk)
None


In [15]:
from ultralytics import YOLO
import torch

# Load YOLOv9 Pose model (s/m/l/x, or your own .pt)
model = YOLO("yolov8s-pose.pt")

In [11]:
if torch.cuda.is_available():
    print("CUDA is available.")
    print(f"Number of CUDA devices: {torch.cuda.device_count()}")
    print(f"Current CUDA device name: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available.")

CUDA is available.
Number of CUDA devices: 1
Current CUDA device name: Tesla V100-PCIE-16GB


close_mosaic=1

Mosaic augmentation mixes 4 images together.

Great for large human pose, but can distort small rigid objects → hurts keypoint accuracy.

close_mosaic=1 disables mosaic augmentation after the first 10 epochs (by default).

🔹 Recommended:

✅ Always set close_mosaic=1 for small rigid tools.

You can even disable it completely with mosaic=0.0 in a custom hyp.yaml.

In [16]:
model.train(
    data="surgical_tools_pose.yaml",
    imgsz=960,      # bigger img size helps small objects (If VRAM allows, go 1280 for best keypoint precision. 
                    # Avoid 640 → too coarse for tiny tools.)
    epochs=300, #200 is a good default for small datasets.
                # If dataset is very small (<5k images), you may need 300–400.
                # Use early_stopping (Ultralytics auto monitors val) to stop if performance plateaus.
    batch=32,   # Larger batch → smoother gradient, but requires more VRAM.
                # Pick the largest batch your GPU fits at chosen imgsz.
                #For 24GB VRAM: batch=32–64 at imgsz=960.
                #For 12GB VRAM: batch=16 is usually safe.
    close_mosaic=1,
    workers=8,
    patience=50  
)

Ultralytics 8.3.179 🚀 Python-3.10.18 torch-2.5.1+cu121 CUDA:0 (Tesla V100-PCIE-16GB, 16144MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=1, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=surgical_tools_pose.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s-pose.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=50, perspective=0.0, plots=True, pose=12.

YOLOv8s-pose summary: 144 layers, 11,423,939 parameters, 11,423,923 gradients, 29.6 GFLOPs

Transferred 355/397 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2770.6±506.3 MB/s, size: 438.9 KB)


train: Scanning /home/student/Synthetic-data/output/train/labels... 1250 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1250/1250 [00:08<00:00, 142.07it/s]

train: New cache created: /home/student/Synthetic-data/output/train/labels.cache
WARNING ⚠️ No 'flip_idx' array defined in data.yaml, disabling 'fliplr' and 'flipud' augmentations.


...

    231/300      14.4G     0.8268     0.9019     0.1239      1.159       1.42         69        960:  12%|█▎        | 5/40 [00:02<00:14,  2.39it/s]libpng warning: iCCP: profile 'ICC Profile': 1000000h: invalid rendering intent
    231/300      14.4G     0.8342     0.8072     0.1411      1.109      1.436         82        960:  48%|████▊     | 19/40 [00:08<00:08,  2.39it/s]libpng warning: iCCP: profile 'ICC Profile': 'GRAY': Gray color space not permitted on RGB PNG
    231/300      14.4G     0.8388      0.815     0.1408      1.109       1.44         83        960:  52%|█████▎    | 21/40 [00:08<00:07,  2.39it/s]libpng warning: iCCP: profile 'ICC Profile': 'desc': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'wtpt': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'bkpt': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'rXYZ': ICC profile tag start not a multiple of 4
li

                   all        150        150      0.477      0.425      0.379      0.173      0.578        0.5      0.511      0.289
EarlyStopping: Training stopped early as no improvement observed in last 50 epochs. Best results observed at epoch 181, best model saved as best.pt.
To update EarlyStopping(patience=50) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

231 epochs completed in 1.219 hours.
Optimizer stripped from runs/pose/train3/weights/last.pt, 23.2MB
Optimizer stripped from runs/pose/train3/weights/best.pt, 23.2MB

Validating runs/pose/train3/weights/best.pt...
Ultralytics 8.3.179 🚀 Python-3.10.18 torch-2.5.1+cu121 CUDA:0 (Tesla V100-PCIE-16GB, 16144MiB)
YOLOv8s-pose summary (fused): 81 layers, 11,413,731 parameters, 0 gradients, 29.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95):   0%|          | 0/3 [00:00<?, ?it/s]libpng warning: iCCP: profile 'ICC Profile': 1000000h: invalid rendering intent
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95):  33%|███▎      | 1/3 [00:00<00:00,  2.06it/s]libpng warning: iCCP: profile 'ICC Profile': 'GRAY': Gray color space not permitted on RGB PNG
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  1.72it/s]


                   all        150        150        0.5      0.438      0.418      0.188      0.534      0.464       0.47      0.313
         needle_holder         90         90      0.614      0.644      0.655      0.307      0.663      0.689       0.73      0.509
              tweezers         60         60      0.386      0.231      0.182     0.0702      0.405      0.238      0.209      0.116
Speed: 0.5ms preprocess, 3.1ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs/pose/train3


ultralytics.utils.metrics.PoseMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f4088667070>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(P)', 'F1-Confidence(P)', 'Precision-Confidence(P)', 'Recall-Confidence(P)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.03403

Warning about flips
WARNING No 'flip_idx' array defined in data.yaml, disabling fliplr/flipud.


If you want horizontal flip augmentation for pose, you must tell the model how your keypoints swap under a left–right flip.
Given your 5 kps are ["tl","tr","br","bl","center"], add this to surgical_tools_pose.yaml:

kpt_shape: [5, 3]
flip_idx: [1, 0, 3, 2, 4]  # tl↔tr, br↔bl, center→center


If flips don’t make sense for your tools, you can leave it disabled (current behavior).